# Reservoir parameterization Hanasaki et al 2006 for mizuRoute


Based on: 
Hanasaki, N., Kanae, S., & Oki, T. (2006). A reservoir operation scheme for global river routing models. Journal of Hydrology, 327(1–2), 22–41. https://doi.org/10.1016/j.jhydrol.2005.11.011

by Shervan Gharari based on the code by Inne Vanderkelen - January 2021

In [1]:
# load the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
# constants
secperday   = 86400
secperhour  = 3600
dt          = 86400 # time of the modeling can be also hourly for mizuRoute
daypermonth = 30 # average day in a month

In [3]:
# control file flags that should be set to true or T
Hanasaki_model   = True # identifies that the Hanasaki model is called at least once in the river network topoloty
Water_management = False # if the hanasaki lake flag Par_flag_D is true then this should be true in the
# control file and the second nc file containting the water magement should be pass to mizuRoute or through
# coupler (when fully develop) any case the lake formualtion expect the demand values of that time steps


In [4]:
# river network topology variables and parameters
# config in the river network topology alog the paraemters a lake type varibale should be specified
# for Hanasaki this is 2 (for Doll it is 1); it tells which parameteric model the model should follow
# for a given lake
lake_model_type  = 2 # Hanasaki; this is not used here for this code just to calrify
Par_flag_purpose = True # true agricultur; false others
Par_flag_I       = False # false use paraemter always, True transition from parameters to 
Par_flag_D       = False # false use paraemter always, True transition from parameters to

In [7]:
def Hanasaki (# Hanasaki inputs
              S, #storage at the current time step
              I, # upstream inflow
              D, # demand provided by the from outside mizuRoute
              # Hanasaki paraemters
              Par_flag_purpose, # porpouse of the lake, True if irrigation
              Par_Smax, # maximume storage of the reservoir
              # inflow
              Par_I_month, # inflow 12 timesteps for each month starting from january
              Par_flag_I, # flag telling to move to simulated inflow by mizuRoute
              # demand
              Par_D_month, # demand 12 timesteps for each month starting from january
              Par_flag_D, # flag telling to move to the provided demand from outside of mizuRoute
              # paraemters/coefficient
              Par_alpha, # storage activity
              Par_envfact, # Environmetal flow requirement
              Par_release_ini, # inital value of release coefficient if simulation not starting at first operational month
              Par_C1, # Coefficient 1
              Par_C2,# coefficient 2
              Par_power, # power
              Par_dinominator, # dinominator
              Par_c_compare,  # for daily release - IV: not clear to me why you use this variable? 
              # state/memory array
              Mem_I, # legth of the memory for inflow in year
              Mem_D, # length of the memory for demand in year
              # simulation paraemters
              yyyy,    # current simulation year 
              mm,      # current simulation month
              dd,      # current simulation day
              hh,      # current simulation hour
              mi,      # current simulation minute
              ss,      # current simulation second
              i):      # time steps of simulation; i = 1 the initial time step of the model
    # IV: should dt not also be passed through the module? 
    
    # calculation of Hansaki dependent parameters
    # yearly average inflow to the reservoir
    if Par_flag_I: # starting from inflow paraemter and transition to the mizuRoute inflow to lakes 
        if np.isnan(Mem_I): # if memory is nan it should be allocated [equivalant of allcation in Fortran]
            # 12 month and the number of time steps to be saved from the past
            Mem_I = np.zeros ([12,np.floor(secperday/dt*daypermonth*Par_Mem_I)])
            for i in np.arange(12):
                Mem_I[i,:] = Mem_I[i,:]+Par_I_month[i] # populate the 2D vector for the model
        else: # is allocated so shift the Mem_I and give the inflow, I, to the proper index
            # shift one time step and put the value in its right place
            Mem_I [:,1:] = Mem_I [:,:-1]
            Mem_I [mm-1,0] = I
        # calculate the yearly input
        I_yearly = Mem_I.mean()
        Par_I_month = Mem_I.mean(axis=1) # update the paraemters field 
    else:
        I_yearly = Par_I_month.mean()
    
    # yearly average demand to the reservoir
    if Par_flag_D: # starting from demand parameter and transition to the mizuRoute demand to lakes
        if not Water_management:
            print('need demand values Water_management flag should be true')
            sys.exit()
        if np.isnan(Mem_D): # if memory is nan it should be allocated [equivalant of allcation in Fortran]
            # 12 month and the number of time steps to be saved from the past
            Mem_D = np.zeros ([12,np.floor(secperday/dt*daypermonth*Par_Mem_D)])
            for i in np.arange(12):
                Mem_D[i,:] = Mem_D[i,:]+Par_D_month[i] # populate the 2D vector for the model
        else: # is allocated so shift the Mem_I and give the inflow, I, to the proper index
            # shift one time step and put the value in its right place
            Mem_D [:,1:] = Mem_D [:,:-1]
            Mem_D [mm-1,0] = D
        # calculate the yearly input
        D_yearly = Mem_D.mean()
        Par_D_month = Mem_D.mean(axis=1) # update the paraemters field 
    else:
        D_yearly = Par_D_month.mean()
        
    # calculate storage to yearly activity ratio
    c = Par_Smax / (I_yearly * secperhour)
    
    # finding the start month of the operational year and release
    E_release = Par_release_ini # IV: can't we here use the initial S passed through the module (how is that set currently in the model?)
    # to detemine E_release in the first months before the first operational year starts? (e.g. using an if statement for i == 1? )  
    # Now, this value gets overwritten every timestep. 
    
    temp = np.arange(12) # index 0 to 11
    temp = temp [::-1] # reverse
    for i in temp:
        if I_yearly <= Par_I_month[i]: # finding the first month
            start_month = i+1 # start of the month
    if mm == start_month:
        E_release = S / (Par_Smax*Par_alpha); # replace release from inital value to actual value
        
    # Determine the target monthly release
    for i in np.arange(12):     # IV: why do you loop here over the different months? Can't you just select monthly mean outflow (and demand) based on the current month (mm)? 
    # i.e. replace i index with mm
        if Par_flag_purpose:
            if Par_envfact * I_yearly <= D_yearly:
                target = Par_I_month[i] * Par_C1 + I_yearly * (Par_D_month[i]/D_yearly) * Par_C1
            else:
                target = I_yearly + Par_D_month[i] - D_yearly
        else:
            target = I_yearly
        
    # determine the actual reservoir release
    if c <= 0.5:
        release_daily = target * E_release
    else:
        release_daily = (c/Par_dinominator)**Par_power*E_release + (1-(c/Par_dinominator)**Par_power)* I
    
    
        
    

In [2]:
# IV add on: putting in reservoir constraints on reservoir storage into the Hanasaki module:


# 1. adjust release so that reservoir does not empty (S goes below dead storage, can be defined as 0.1 of maximum storage, e.g. Biemans 2011, or as 0)
# this introduces yet another parameter though ... What do you think? 
# I see you already accounted for this in the lake code
#
if S < Par_Smax * Par_deadstor: # if reservoir storage falls below the dead storage, subtract the difference from the daily release value 
    release_daily = release_daily - (Par_Smax * Par_deadstor - S) * dt 
    
# 2. Adjust release to account for spilling releases (storage exceeds max storage, all abundant water is released)    
elif S > Par_Smax: 
    release_daily = release_daily + (S - Par_Smax) * dt

# prior to these statements, the storage needs to be adjusted of course, this is not yet in the code above. 
# do you see it is best to derive storage within the Hanasaki routine or would you only use it to derive the outflow values?      
